In [ ]:
# import packages
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import label_binarize
from sklearn.metrics import (roc_curve, roc_auc_score, auc, log_loss,
                             precision_score, recall_score, f1_score,
                             accuracy_score, classification_report,
                             ConfusionMatrixDisplay, confusion_matrix)


import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

#Tensorflow
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.layers import (Dense, Dropout,BatchNormalization, Input, Conv1D, Flatten,
                             MaxPooling1D)
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

import cv2
import os
from skimage.measure import label
from skimage.transform import resize
from skimage.segmentation import slic, mark_boundaries
from skimage.color import label2rgb

#Scikit_learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import (roc_curve, roc_auc_score, auc, log_loss,
                             precision_score, recall_score, f1_score,
                             accuracy_score, classification_report,
                             ConfusionMatrixDisplay, confusion_matrix)
from sklearn import metrics
from sklearn.preprocessing import label_binarize

## Two classifier models based on Zernike moments (ZMs):

In [ ]:
from ZEMO import zemo
import cv2

def calculate_zernike_moments(data_dir, image_size, zernike_order):
        
        ZBFSTR = zemo.zernike_bf(image_size, zernike_order, 1)
        
        image_files = [os.path.join(data_dir, filename) for filename in os.listdir(data_dir) if filename.endswith('.jpg')]
        
        zernike_moments = []
    
        for img_path in image_files:
            image = cv2.imread(img_path)
            resized_image = cv2.resize(image, (image_size,image_size))
            im = resized_image[:, :, 0]
            Z = np.abs(zemo.zernike_mom(np.array(im), ZBFSTR))
            zernike_moments.append(Z)
        
        df = pd.DataFrame(zernike_moments)
    
        return df

In [ ]:
s1path = r'/remote-home/cs_acmis_hby/Galaxy-Zoo-Classification/Contrast_experiment/gz2_images_dataset07/barred_spirals' 
s2path = r'/remote-home/cs_acmis_hby/Galaxy-Zoo-Classification/Contrast_experiment/gz2_images_dataset07/unbarred_spirals'

e1path = r'/remote-home/cs_acmis_hby/Galaxy-Zoo-Classification/Contrast_experiment/gz2_images_dataset07/round_elliptical'
e2path = r'/remote-home/cs_acmis_hby/Galaxy-Zoo-Classification/Contrast_experiment/gz2_images_dataset07/cigar_shaped_elliptical'
e3path = r'/remote-home/cs_acmis_hby/Galaxy-Zoo-Classification/Contrast_experiment/gz2_images_dataset07/in_between_elliptical'

o1path = r'/remote-home/cs_acmis_hby/Galaxy-Zoo-Classification/Contrast_experiment/gz2_images_dataset07/irregular' 
o2path = r'/remote-home/cs_acmis_hby/Galaxy-Zoo-Classification/Contrast_experiment/gz2_images_dataset07/merger' 



# Defult image size and zernike order. 
image_size = 224
zernike_order = 45

spiral_zm_df1 = calculate_zernike_moments(s1path, image_size, zernike_order)
spiral_zm_df1.to_csv('/remote-home/cs_acmis_hby/Galaxy-Zoo-Classification/Contrast_experiment/ZMs/barred_spirals_zms.csv')

spiral_zm_df2= calculate_zernike_moments(s2path, image_size, zernike_order)
spiral_zm_df2.to_csv('/remote-home/cs_acmis_hby/Galaxy-Zoo-Classification/Contrast_experiment/ZMs/unbarred_spirals_zms.csv')



elliptical_zm_df1 = calculate_zernike_moments(e1path, image_size, zernike_order)
elliptical_zm_df1.to_csv('/remote-home/cs_acmis_hby/Galaxy-Zoo-Classification/Contrast_experiment/ZMs/round_elliptical_zms.csv')

elliptical_zm_df2 = calculate_zernike_moments(e2path, image_size, zernike_order)
elliptical_zm_df2.to_csv('/remote-home/cs_acmis_hby/Galaxy-Zoo-Classification/Contrast_experiment/ZMs/cigar_shaped_elliptical_zms.csv')

elliptical_zm_df3 = calculate_zernike_moments(e3path, image_size, zernike_order)
elliptical_zm_df3.to_csv('/remote-home/cs_acmis_hby/Galaxy-Zoo-Classification/Contrast_experiment/ZMs/in_between_elliptical_zms.csv')

odd_zm_df1 = calculate_zernike_moments(o1path, image_size, zernike_order)
odd_zm_df1.to_csv('/remote-home/cs_acmis_hby/Galaxy-Zoo-Classification/Contrast_experiment/ZMs/irregular_zms.csv')

odd_zm_df2 = calculate_zernike_moments(o2path, image_size, zernike_order)
odd_zm_df2.to_csv('/remote-home/cs_acmis_hby/Galaxy-Zoo-Classification/Contrast_experiment/ZMs/merger_zms.csv')

np.shape(spiral_zm_df1), np.shape(elliptical_zm_df1), np.shape(odd_zm_df1)

In [ ]:
# Defult image size and zernike order. 
image_size = 224
zernike_order = 45
edge_on_zm_df = calculate_zernike_moments('/remote-home/cs_acmis_hby/Galaxy-Zoo-Classification/Contrast_experiment/gz2_images_dataset07/edge_on', image_size, zernike_order)
elliptical_zm_df1.to_csv('/remote-home/cs_acmis_hby/Galaxy-Zoo-Classification/Contrast_experiment/ZMs/edge_on_zms.csv')

# data for ZMs

In [ ]:
spiral_data1 = pd.read_csv('/remote-home/cs_acmis_hby/Galaxy-Zoo-Classification/Contrast_experiment/ZMs/barred_spirals_zms.csv')
spiral_data2 = pd.read_csv('/remote-home/cs_acmis_hby/Galaxy-Zoo-Classification/Contrast_experiment/ZMs/unbarred_spirals_zms.csv')

edge_on_data = pd.read_csv('/remote-home/cs_acmis_hby/Galaxy-Zoo-Classification/Contrast_experiment/ZMs/round_elliptical_zms.csv')

elliptical_data1 = pd.read_csv('/remote-home/cs_acmis_hby/Galaxy-Zoo-Classification/Contrast_experiment/ZMs/round_elliptical_zms.csv')
elliptical_data2 = pd.read_csv('/remote-home/cs_acmis_hby/Galaxy-Zoo-Classification/Contrast_experiment/ZMs/cigar_shaped_elliptical_zms.csv')
elliptical_data3 = pd.read_csv('/remote-home/cs_acmis_hby/Galaxy-Zoo-Classification/Contrast_experiment/ZMs/in_between_elliptical_zms.csv')


odd_data1 = pd.read_csv('/remote-home/cs_acmis_hby/Galaxy-Zoo-Classification/Contrast_experiment/ZMs/irregular_zms.csv')
odd_data2 = pd.read_csv('/remote-home/cs_acmis_hby/Galaxy-Zoo-Classification/Contrast_experiment/ZMs/merger_zms.csv')




spiral_data1.drop("Unnamed: 0", axis = 1, inplace = True)
spiral_data2.drop("Unnamed: 0", axis = 1, inplace = True)

edge_on_data.drop("Unnamed: 0", axis = 1, inplace = True)


elliptical_data1.drop("Unnamed: 0", axis = 1, inplace = True)
elliptical_data2.drop("Unnamed: 0", axis = 1, inplace = True)
elliptical_data3.drop("Unnamed: 0", axis = 1, inplace = True)

odd_data1.drop("Unnamed: 0", axis = 1, inplace = True)
odd_data2.drop("Unnamed: 0", axis = 1, inplace = True)

all_zm_data = np.concatenate([spiral_data1,spiral_data2,edge_on_data, elliptical_data1,elliptical_data2,elliptical_data3, odd_data1,odd_data2])
np.shape(all_zm_data)

In [ ]:
spiral_label0 = [0] * len(spiral_data1)
spiral_label1 = [1] * len(spiral_data2)

elliptical_label2 = [2] * len(elliptical_data1)
elliptical_label3 = [3] * len(elliptical_data2)
elliptical_label4 = [4] * len(elliptical_data3)

edge_on_label5 = [5] * len(edge_on_data)

odd_label6 = [6] * len(odd_data1)
odd_label7 = [7] * len(odd_data2)


all_labels = np.concatenate([spiral_label0,spiral_label1, elliptical_label2,elliptical_label3,elliptical_label4,edge_on_label5, odd_label6,odd_label7])
len(all_labels)

# **SVM + ZMs**

In [ ]:
X_train, X_test, y_train, y_test, train_indices, test_indices = train_test_split(all_zm_data, all_labels, np.arange(len(all_labels)), 
                                                                                 test_size=0.2, shuffle=True, random_state=None)

In [ ]:
print(train_indices)

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
model = SVC(kernel='rbf', probability=True, C=1.5, gamma='scale')
gz2_training_model = model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

con = metrics.confusion_matrix(y_test, y_pred)
print(con)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics

# 类别名称
class_names = [
    'Barred Spiral', 
    'Unbarred Spiral', 
    'Round Elliptical', 
    'Cigar Elliptical', 
    'In-between Elliptical', 
    'Edge-on Disk', 
    'Irregular', 
    'Merger'
]

# ✨【重点】先转 numpy array！
y_pred = np.array(y_pred)
y_test = np.array(y_test)

# ✨判断如果是概率数组，就取最大值索引
if y_pred.ndim > 1:
    y_pred_labels = np.argmax(y_pred, axis=1)
else:
    y_pred_labels = y_pred

# 混淆矩阵
con = metrics.confusion_matrix(y_test, y_pred_labels)

# 输出 classification_report
print(metrics.classification_report(y_test, y_pred_labels, target_names=class_names, digits=4))

# 画混淆矩阵
plt.figure(figsize=(10, 8))
sns.heatmap(con, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Label', fontsize=12)
plt.ylabel('True Label', fontsize=12)
plt.title('Confusion Matrix (Counts)', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# 画归一化混淆矩阵
con_norm = con.astype('float') / con.sum(axis=1)[:, np.newaxis]
plt.figure(figsize=(10, 8))
sns.heatmap(con_norm, annot=True, fmt='.2f', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Label', fontsize=12)
plt.ylabel('True Label', fontsize=12)
plt.title('Normalized Confusion Matrix (Proportion)', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()


# 1D_CNN + ZMs

In [ ]:
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, MaxPooling1D, Dropout, Flatten, Dense
from tensorflow.keras.models import Model
import tensorflow as tf

# 输入层
x = Input(shape=(all_zm_data.shape[1],1))

# 隐藏层（保持不变）
c0 = Conv1D(256, kernel_size=3, strides=2, padding="same")(x)
b0 = BatchNormalization()(c0)
m0 = MaxPooling1D(pool_size=2)(b0)
d0 = Dropout(0.1)(m0)

c1 = Conv1D(128, kernel_size=3, strides=2, padding="same")(d0)
b1 = BatchNormalization()(c1)
m1 = MaxPooling1D(pool_size=2)(b1)
d1 = Dropout(0.1)(m1)

c2 = Conv1D(64, kernel_size=3, strides=2, padding="same")(d1)
b2 = BatchNormalization()(c2)
m2 = MaxPooling1D(pool_size=2)(b2)
d2 = Dropout(0.1)(m2)

f = Flatten()(d2)

# 输出层（修改为8分类）
de0 = Dense(64, activation='relu')(f)
de1 = Dense(32, activation='relu')(de0)
de2 = Dense(8, activation='softmax')(de1)  # 8个输出神经元

# 创建模型
model = Model(inputs=x, outputs=de2, name="cnn_zm_8class_galaxy")
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()
model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
model.summary()

In [ ]:
spiral_label0 = [0] * len(spiral_data1)
spiral_label1 = [1] * len(spiral_data2)

elliptical_label2 = [2] * len(elliptical_data1)
elliptical_label3 = [3] * len(elliptical_data2)
elliptical_label4 = [4] * len(elliptical_data3)

edge_on_label5 = [5] * len(edge_on_data)

odd_label6 = [6] * len(odd_data1)
odd_label7 = [7] * len(odd_data2)


all_labels = np.concatenate([spiral_label0,spiral_label1, elliptical_label2,elliptical_label3,elliptical_label4,edge_on_label5, odd_label6,odd_label7])
len(all_labels)

In [ ]:
# Callback Function
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

b_size = 64
e_num = 30

history = model.fit(
X_train, y_train_encoded,
batch_size=b_size,
epochs=e_num,
# class_weight=class_weights,
    
verbose = 1,
callbacks=es,
validation_split=0.01)

In [ ]:
y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)

con = metrics.confusion_matrix(y_test, y_pred_labels)
print(con)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics

# 类别名称
class_names = [
    'Barred Spiral', 
    'Unbarred Spiral', 
    'Round Elliptical', 
    'Cigar Elliptical', 
    'In-between Elliptical', 
    'Edge-on Disk', 
    'Irregular', 
    'Merger'
]

# ✨【重点】先转 numpy array！
y_pred = np.array(y_pred_labels)
y_test = np.array(y_test)

# ✨判断如果是概率数组，就取最大值索引
if y_pred.ndim > 1:
    y_pred_labels = np.argmax(y_pred, axis=1)
else:
    y_pred_labels = y_pred

# 混淆矩阵
con = metrics.confusion_matrix(y_test, y_pred_labels)

# 输出 classification_report
print(metrics.classification_report(y_test, y_pred_labels, target_names=class_names, digits=4))

# 画混淆矩阵
plt.figure(figsize=(10, 8))
sns.heatmap(con, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Label', fontsize=12)
plt.ylabel('True Label', fontsize=12)

plt.title('Confusion Matrix (Counts)', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# 画归一化混淆矩阵
con_norm = con.astype('float') / con.sum(axis=1)[:, np.newaxis]
plt.figure(figsize=(10, 8))
sns.heatmap(con_norm, annot=True, fmt='.2f', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Label', fontsize=12)
plt.ylabel('True Label', fontsize=12)
plt.title('Normalized Confusion Matrix (Proportion)', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()
